In [1]:
import matplotlib.pyplot as plt
import keras
from keras import optimizers
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
import tensorflow as tf
import os
import glob
import PIL
import numpy as np 
import cv2
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns
from jupyterthemes import jtplot
import innvestigate
import shap
from deepexplain.tensorflow import DeepExplain
from sklearn.preprocessing import normalize
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)
from PIL import *
import matplotlib.cm as cm
%matplotlib inline
K.clear_session()

Using TensorFlow backend.
/home/amitoj/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/amitoj/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/amitoj/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/amitoj/anaconda3/lib/python3.7/site-packages/tenso

In [2]:
#base = "/../../../scratch/amitoj/OCT2017/"
base = "../../hdd/Amitoj/OCT2017/"

In [3]:
x_test = np.load('annot_keras.npy')

In [4]:
x_test.shape

(4, 10, 299, 299, 3)

In [5]:
y_test = np.array([[0 for i in range(x_test.shape[0])], [1 for i in range(x_test.shape[0])], [2 for i in range(x_test.shape[0])], [3 for i in range(x_test.shape[0])]])

In [6]:
model = InceptionV3(include_top=True, weights= None, classes=4)
adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, decay=0.0, amsgrad=False)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
weight_addrs = glob.glob('../Kermany_inception/top_weights/*')

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
weight_addrs.sort()
weight_addrs

['../Kermany_inception/top_weights/inceptionV3-02-0.9910.hdf5',
 '../Kermany_inception/top_weights/inceptionV3-03-0.9960.hdf5',
 '../Kermany_inception/top_weights/inceptionV3-04-0.9900.hdf5',
 '../Kermany_inception/top_weights/inceptionV3-04-0.9960.hdf5',
 '../Kermany_inception/top_weights/inceptionV3-04-1.0000.hdf5',
 '../Kermany_inception/top_weights/inceptionV3-06-0.9930.hdf5',
 '../Kermany_inception/top_weights/inceptionV3-08-0.9970.hdf5',
 '../Kermany_inception/top_weights/inceptionV3-09-0.9930.hdf5',
 '../Kermany_inception/top_weights/inceptionV3-10-0.9930.hdf5',
 '../Kermany_inception/top_weights/inceptionV3-16-0.9990.hdf5']

In [7]:
x = np.load(base + 'small_train_norm.npy', allow_pickle = True)

In [8]:
# background for shap
#load background distribution
randomlist = [479, 922, 491, 450, 1017, 1295, 988, 240, 809, 904, 621, 20, 141, 366, 373, 1110, 1003, 1149, 135, 94]
x_train = np.empty((x.shape[0], 299, 299,3))
for i, img in enumerate(x):
    x_small = cv2.cvtColor(cv2.resize(img, dsize=(299,299), 
                                            interpolation=cv2.INTER_CUBIC), cv2.COLOR_GRAY2RGB)
    x_train[i] = x_small
background = x_train[randomlist]
# Create explainer using this background
# explain predictions of the model on three images

In [9]:
keras.backend.get_session().run(tf.global_variables_initializer())
for exp in ['shap_rand']:
    print("\n")
    for w_num, w in enumerate(weight_addrs):
        arr = []
        print("Loading " + w)
        model.load_weights(w)
        pred_arr = []
        for cl in range(3):
            pred_arr.append(model.predict(x_test[cl]))
        #pred_arr
        Y_pred = np.argmax(np.array(pred_arr), axis = 2)
        print("Creating " + str(exp) + " explainer...")
        e = shap.DeepExplainer(model, background)
    
        for cat in range(3): # over 4 classes
            # predicting for the images in the ind_img arr
            print("Finding " + exp + " for class " + str(cat))
            a_curr = []
            sh = e.shap_values(x_test[cat][:]) #loading shap values for the correct class, sh is a list of len 4
            sh_arr = np.array(sh)
            print(sh_arr.shape)
            sh_new = []
            for ind in range(sh_arr.shape[1]):
                sh_new.append(sh_arr[Y_pred[cat][ind]][ind])
            sh = np.array(sh_new)
            sv = np.sum(sh, axis = 3) # adding for all 3 channels
            sv /= (np.abs(sv).max())
            print(sv.shape)
            arr.append(sv) # arr will have sv of all 4 classes
        arr = np.array(arr)
        print(arr.shape)
        np.save('OCT-annotations/attributions_10_all/' + exp + '_model_' + str(w_num), arr)



Loading ../Kermany_inception/top_weights/inceptionV3-02-0.9910.hdf5
Creating shap_rand explainer...


keras is no longer supported, please use tf.keras instead.


FailedPreconditionError: Attempting to use uninitialized value batch_normalization_38/moving_variance
	 [[node batch_normalization_38/moving_variance/read (defined at /home/amitoj/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:402) ]]
	 [[node predictions/Softmax (defined at /home/amitoj/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3231) ]]

Caused by op 'batch_normalization_38/moving_variance/read', defined at:
  File "/home/amitoj/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/amitoj/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/amitoj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/amitoj/anaconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/amitoj/anaconda3/lib/

In [22]:
#'guided_backprop', 'deep_taylor', 'input_t_gradient', 'lrp.z', 'lrp.epsilon', 'integrated_gradients', 'deconvnet'
explainers = ['guided_backprop', 'deep_taylor', 'input_t_gradient', 'lrp.z', 'lrp.epsilon', 'integrated_gradients', 'deconvnet', 'smoothgrad', 'gradient']

In [23]:
for exp in explainers:
    print("\n")
    for w_num, w in enumerate(weight_addrs[0:1]):
        arr = []
        print("Loading " + w)
        model.load_weights(w)
        print("Creating " + str(exp) + "explainer...")
        analyzer = innvestigate.create_analyzer(exp, innvestigate.utils.keras.graph.model_wo_softmax(model))
    
        for cat in range(3): # over 4 classes
            # predicting for the images in the ind_img arr
            print("Finding " + exp + "for class " + str(cat))
            a_curr = []
            for ind in range(0,x_test.shape[1],1):
                a = analyzer.analyze(x_test[cat][ind:ind+1])
            # Aggregate along color channels and normalize to [-1, 1]
                a = a.sum(axis=np.argmax(np.asarray(a.shape) == 3))
                a /= np.max(np.abs(a))
                a_curr.append(a[0])
            a_c = np.array(a_curr)
            arr.append(np.asarray(a_c))        
        arr = np.array(arr)
        print(arr.shape)
        np.save('OCT-annotations/attributions_20_new/' + exp + '_model_' + str(w_num), arr)



Loading ../Kermany_inception/top_weights/inceptionV3-02-0.9910.hdf5
Creating guided_backpropexplainer...
Finding guided_backpropfor class 0
Finding guided_backpropfor class 1
Finding guided_backpropfor class 2
(3, 20, 299, 299)


Loading ../Kermany_inception/top_weights/inceptionV3-02-0.9910.hdf5
Creating deep_taylorexplainer...
Finding deep_taylorfor class 0
Finding deep_taylorfor class 1
Finding deep_taylorfor class 2
(3, 20, 299, 299)


Loading ../Kermany_inception/top_weights/inceptionV3-02-0.9910.hdf5
Creating input_t_gradientexplainer...
Finding input_t_gradientfor class 0
Finding input_t_gradientfor class 1
Finding input_t_gradientfor class 2
(3, 20, 299, 299)


Loading ../Kermany_inception/top_weights/inceptionV3-02-0.9910.hdf5
Creating lrp.zexplainer...
Finding lrp.zfor class 0
Finding lrp.zfor class 1
Finding lrp.zfor class 2
(3, 20, 299, 299)


Loading ../Kermany_inception/top_weights/inceptionV3-02-0.9910.hdf5
Creating lrp.epsilonexplainer...
Finding lrp.epsilonfor class 

In [ ]:
x = np.load('train_select_keras.npy')
background = x

In [ ]:
for exp in ['shap_select']:
    print("\n")
    for w_num, w in enumerate(weight_addrs):
        arr = []
        print("Loading " + w)
        model.load_weights(w)
        pred_arr = []
        for cl in range(3):
            pred_arr.append(model.predict(x_test[cl]))
        #pred_arr
        Y_pred = np.argmax(np.array(pred_arr), axis = 2)
        print("Creating " + str(exp) + " explainer...")
        e = shap.DeepExplainer(model, background)
    
        for cat in range(3): # over 4 classes
            # predicting for the images in the ind_img arr
            print("Finding " + exp + " for class " + str(cat))
            a_curr = []
            sh = e.shap_values(x_test[cat][:]) #loading shap values for the correct class, sh is a list of len 4
            sh_arr = np.array(sh)
            print(sh_arr.shape)
            sh_new = []
            for ind in range(sh_arr.shape[1]):
                sh_new.append(sh_arr[Y_pred[cat][ind]][ind])
            sh = np.array(sh_new)
            sv = np.sum(sh, axis = 3) # adding for all 3 channels
            sv /= (np.abs(sv).max())
            print(sv.shape)
            arr.append(sv) # arr will have sv of all 4 classes
        arr = np.array(arr)
        print(arr.shape)
        np.save('OCT-annotations/attributions_20_new/' + exp + '_model_' + str(w_num), arr)

In [19]:
explainers = ['saliency', 'deeplift', 'occlusion']

In [21]:
import keras
ind_img = [i for i in range(0,250,1)] # selecting images from each class
for exp in explainers:
    print("\n")
    for w_num, w in enumerate(weight_addrs[0:1]):
        arr = []
        print("Loading " + w)
        model.load_weights(w)
        
        pred_arr = []
        for cl in range(3):
            pred_arr.append(model.predict(x_test[cl]))
        Y_pred = np.argmax(np.array(pred_arr), axis = 2)
        
        print("Creating " + str(exp) + " explainer...")
        input_tensor = model.layers[0].input
        fModel = Model(inputs=input_tensor, outputs = model.layers[-1].output) #excluding dense layer
        target_tensor = fModel(input_tensor)
        for cat in range(3): # over 4 classes
            # predicting for the images in the ind_img arr
            print("Finding " + exp + " for class " + str(cat))
            with DeepExplain(session=K.get_session()) as de:
                xs = x_test[cat]
                ys = keras.utils.to_categorical(Y_pred[cat], 4)
                if(exp == 'occlusion'):
                    a = de.explain('occlusion', target_tensor, input_tensor, xs, ys=ys, window_shape=[64,64,3],  step=16, batch_size=40)
                    a = np.nan_to_num(a)
                else:
                    a = de.explain(exp, target_tensor, input_tensor, xs, ys=ys, batch_size=40)
                a = a.sum(axis=np.argmax(np.asarray(a.shape) == 3))
                a /= np.max(np.abs(a))
            a_c = np.array(a)
            arr.append(np.asarray(a_c))
            print(a_c.shape)
        arr = np.array(arr)
        print(arr.shape)
        np.save('OCT-annotations/attributions_20_new//' + exp + '_model_' + str(w_num), arr)



Loading ../Kermany_inception/top_weights/inceptionV3-02-0.9910.hdf5
Creating saliency explainer...
Finding saliency for class 0
(20, 299, 299)
Finding saliency for class 1
(20, 299, 299)
Finding saliency for class 2
(20, 299, 299)
(3, 20, 299, 299)


Loading ../Kermany_inception/top_weights/inceptionV3-02-0.9910.hdf5
Creating deeplift explainer...
Finding deeplift for class 0
(20, 299, 299)
Finding deeplift for class 1
(20, 299, 299)
Finding deeplift for class 2
(20, 299, 299)
(3, 20, 299, 299)


Loading ../Kermany_inception/top_weights/inceptionV3-02-0.9910.hdf5
Creating occlusion explainer...
Finding occlusion for class 0


invalid value encountered in true_divide
Attributions generated by Occlusion method contain nans, probably because window_shape and step do not allow to cover the all input.


(20, 299, 299)
Finding occlusion for class 1
(20, 299, 299)
Finding occlusion for class 2
(20, 299, 299)
(3, 20, 299, 299)


In [9]:
print("hello")

hello
